In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from shapely import wkt
import datetime as dt

In [2]:
#La principal caracteristica de esta funcion es permitir ocultar o mostrar ciertas columnas.
#Por si sola NO elimina columnas del dataframe (podria pero no es para lo que esta desarrollada).
"""
name: update_columns
        function to modify a list asociated to a dataframe columns.
        focus on developing tasks.
args:
    - func(str):
            'add'    add a column to the dataframe render
            'del'    delete a column from the dataframe render
    - col_name(str): column name being added/deleted
    - col_list(list): list where the modifications are done
"""
def update_columns(func:str, col_name:str, col_list:list):
    if func == 'add' and col_name not in col_list:
        col_list.append(col_name)
    elif func == 'del' and col_name in col_list:
        col_list.remove(col_name)

In [3]:
"""
name: split_coordinates
    split the wkt coordinates from one column to two new columns with each one of its components.
    new columns has by name sufix_x and sufix_y.
args:
    - col_name(str): name of the column being split
    - dataframe(pd.dataFrame): dataframe where the changes will be stored
    - new_sufix(str):
            default ""    if default, the sufix of the new columns are the original column name.
                          else, new_sufix will be a custom string defined by user.
    - inplace(bool):
            default True    if True, the function doesn't return a dataframe, and changes will be done inplace.
                            if False, the function returns a new-memory allocated dataframe.
                            
return: This function has only return value if inplace is False, in that case, a pd.dataFrame is returned.
"""
def split_coordinates(col_name:str, dataframe, new_sufix:str="", inplace:bool=True):
    new_sufix = col_name if new_sufix == "" else new_sufix
    dataframe[new_sufix + "_x"] = [float(re.findall(r"\d{1,2}\.\d*", x)[0]) for x in dataframe[col_name]]
    dataframe[new_sufix + "_y"] = [float(re.findall(r"\d{1,2}\.\d*", x)[1]) for x in dataframe[col_name]]
    if not(inplace):
        return dataframe.copy()

    
"""
name: split_datetime
    split the datetime column into date and time columns.
args:
    - col_name(str): name of the column being split
    - dataframe(pd.dataFrame): dataframe where the changes will be stored
    - inplace(bool):
            default True    if True, the function doesn't return a dataframe, and changes will be done inplace.
                            if False, the function returns a new-memory allocated dataframe.
                            
return: This function has only return value if inplace is False, in that case, a pd.dataFrame is returned.
"""
def split_datetime(col_name:str, dataframe, inplace:bool=True):
    dataframe["date"] = [ (dt.strptime(x, "%Y-%m-%d %H:%M:%S")).date() for x in dataframe[col_name]]
    dataframe["time"] = [(dt.strptime(x, "%Y-%m-%d %H:%M:%S")).time() for x in dataframe[col_name]]
    if not(inplace):
        return dataframe.copy()


In [4]:
"""
name: read_data
    read and create a new instance of pandas dataframe.
args:
    - filename(str): filename of the .csv file to initialize a dataframe(.csv mandatory file).

return: pandas dataframe with all data in csv.
"""
def read_data(filename:str):
    df_raw_data = pd.read_csv(filename)
    df_raw_data.datetime = pd.to_datetime(df_raw_data["datetime"])
    df_raw_data["eval_minutes"] = [x.hour*60 + x.minute for x in df_raw_data.datetime]
    df_raw_data["hour"] = [x.hour for x in df_raw_data.datetime]
    return df_raw_data

"""
name: mean_by_bbox_region
    calculate the average trips per week for the region asked.
args:
    - df(pd.dataframe): pandas dataframe with split columns for x and y components
    - x_min(float): x component for min point of the bounding box
    - y_min(float): y component for min point of the bounding box
    - x_max(float): x component for max point of the bounding box
    - y_max(float): y component for max point of the bounding box
    - reg(str): name of the region which will be consulting.

return: float value with the mean trips per week.
"""
#first version: purely pandas-local version
def mean_by_bbox_region(df, x_min:float, y_min:float, x_max:float, y_max:float, reg:str):
    mask =  (min_x <= df["origin_x"]) & (df["origin_x"] <= max_x) &\
            (min_y <= df["origin_y"]) & (df["origin_y"] <= max_y) &\
            (min_x <= df["dest_x"]) & (df["dest_x"] <= max_x) &\
            (min_y <= df["dest_y"]) & (df["dest_y"] <= max_y) &\
            (df["region"] == reg)

    df_grouped = df[mask].groupby([pd.Grouper(key="region"), 
                                   pd.Grouper(key="datetime", freq="W")]).origin_x
    total_occ = df_grouped.count().sum()
    total_weeks = len(df_grouped.count())
    return total_occ/total_weeks

In [5]:
df = read_data('trips.csv')
df.drop("datasource", axis=1, inplace=True)
split_coordinates("origin_coord", df, "origin")
split_coordinates("destination_coord", df, "dest")

In [6]:
min_x, min_y, max_x, max_y = (9, 50, 120, 130) #ref values just for testing the function
mean_by_bbox_region(df, min_x, min_y, max_x, max_y, "Hamburg")

5.6

In [7]:
df_to_add = df[["region", "hour", "origin_coord", "destination_coord", "datetime"]].copy()
df_to_add.head()

,region,hour,origin_coord,destination_coord,datetime
0,Prague,9,POINT (14.4973794438195 50.00136875782316),POINT (14.43109483523328 50.04052930943246),2018-05-28 09:03:40
1,Turin,2,POINT (7.672837913286881 44.9957109242058),POINT (7.720368637535126 45.06782385393849),2018-05-21 02:54:04
2,Prague,8,POINT (14.32427345662177 50.00002074358429),POINT (14.47767895969969 50.09339790740321),2018-05-13 08:52:25
3,Turin,9,POINT (7.541509189114433 45.09160503827746),POINT (7.74528653441973 45.02628598341506),2018-05-06 09:49:16
4,Turin,12,POINT (7.614078119815749 45.13433106465422),POINT (7.527497142312585 45.03335051325654),2018-05-23 12:45:54


In [8]:
#I split the local/pandas code from mysql code just for conveniance, can be restructured.
import pymysql
import config
import mysql.connector as mysqlconn

"""
name: make_connection
    Function to stablish a connectino with database accord config.py file credentials.
args: this function doesn't have arguments, but requires credentials defined in config.py
return: return mysql connection interface
"""
def make_connection():
    connection = pymysql.connect(host=config.host,
                             user=config.user,
                             password=config.password,
                             db=config.database)
    return connection


"""
name: clear_table_db
    this developer-intended function allows to clear all the register in the given table from database.
    NOTE: DO NOT USE IN PRODUCTION ENVIRONMENT!!!
args:
    - db_name(str): database name
    - table_name(str): table name
    
return: this function doesn't have a return value
"""
#function just for developing work, delete it after pass to production
def clear_table_db(db_name:str, table_name:str):
    connection = make_connection()
    cursor = connection.cursor()
    cursor.execute(f"use {db_name};")
    sql = f"DELETE FROM `{table_name}`"
    cursor.execute(sql)
    connection.commit()
    connection.close()

"""
name: show_all_records
    Function that retrieves all records from a table, just for visualization.
args:
    - db_name(str): database_name
    - table_name(str): table_name

return: this function doesn't have any return value
"""
# Execute query
def show_all_records(db_name:str, table_name:str):
    connection = make_connection()
    cursor = connection.cursor()
    cursor.execute(f"use {db_name};")
    sql = f"SELECT * FROM {table_name};"
    cursor.execute(sql)

    # Fetch all the records
    result = cursor.fetchall()
    for i in result:
        print(i)
    connection.close()

"""
name: get_data

args:
    - db_name(str): database name
    - table_name(str): table name
    - region(tuple): 
            default ()    if default, all regions will be consider
                          else regions in the tuple will be returned
    - hour(tuple):
            default ()    if default, all hour will be consider
                          else hours in the tuple will be returned
    - datetime(tuple):
            default ()    if default, all datetimes will be consider
                          if two datetime are given, filter between dates.
                          else ignore the datetime filter.
                          
return: pandas dataframe with the register from de database accord the filters
"""
def get_data(db_name:str, table_name:str, region:tuple=(), hour:tuple=(), datetime:tuple=()):
    connection = mysqlconn.connect(host=config.host, 
                                   database = config.database, 
                                   user=config.user, 
                                   passwd=config.password,
                                   use_pure=True)
    
    sql = f"SELECT * FROM {table_name} "
    
    q_region, q_hour, q_datetime = (len(region), len(hour), len(datetime))
    constraints = []
    if (q_region) == 1:
        constraints.append(f"region = {region[0]} ")
    elif (q_region > 1):
        constraints.append(f"region IN({','.join(region)}) ")
        
    if (q_hour == 1):
        constraints.append(f"hour = {hour[0]} ")
    elif (q_hour > 1):
        constraints.append(f"hour IN({','.join(hour)}) ")
        
    if (q_datetime == 2):
        if datetime[0] > datetime[1]:
            datetime = (datetime[1], datetime[0])
        constraints.append(f"BETWEEN {datetime[0].day}/{datetime[0].month}/{datetime[0].year} AND "+\
                           f"{datetime[1].day}/{datetime[1].month}/{datetime[1].year} ")
    elif (q_datetime != 0):
        print("[!] Warning: 2 datetimes needed in the tuple, ignoring datetimes filter.")
    
    sql += "WHERE " + "and".join(constraints) + ";"
    result_dataFrame = pd.read_sql(sql,connection)
    connection.close() #close the connection
    return result_dataFrame



#------------------------- SOLUCION 2.a -----------------------------
"""
name:get_avg_per_bbox
    calculate the average trips per week given a bounding box and region.
    the datasource for this function is a table in a mysql database.
args:
    - db_name(str): database name
    - table_name(str): table name
    - x_min(float): x component for min point of the bounding box
    - y_min(float): y component for min point of the bounding box
    - x_max(float): x component for max point of the bounding box
    - y_max(float): y component for max point of the bounding box
    - reg(str): name of the region which will be consulting.

return: float value with the mean trips per week.
"""
#Version 2: with database approach
#It can be improved a lot, just a first mysql-approach
# for example, the region filter is applied from sql query, but coords
# still being filtered using pandas (so the local system can be overloaded if
# certain region have so many registers)
def get_avg_per_bbox(db_name:str, table_name:str, min_x:float, min_y:float, max_x:float, max_y:float, region:str):
    
    df = get_data(db_name, table_name, region=(f"'{region}'",))
    split_coordinates("origin_coord", df, "origin")
    split_coordinates("destination_coord", df, "dest")
    
    mask =  (min_x <= df["origin_x"]) & (df["origin_x"] <= max_x) &\
            (min_y <= df["origin_y"]) & (df["origin_y"] <= max_y) &\
            (min_x <= df["dest_x"]) & (df["dest_x"] <= max_x) &\
            (min_y <= df["dest_y"]) & (df["dest_y"] <= max_y) &\
            (df["region"] == region)

    df_grouped = df[mask].groupby([pd.Grouper(key="region"), 
                                   pd.Grouper(key="datetime", freq="W")]).origin_x
    total_occ = df_grouped.count().sum()
    total_weeks = len(df_grouped.count())
    return total_occ/total_weeks



#----------------- SOLUCION 1 ------------------------------------
"""
name: add_data
    Function to add registers to a table accord credentials for mysql connection.
args:
    - db_name(str): database name
    - table_name(str): table name
    - dataframe(pd.dataFrame): dataframe data that will be added to database
    
return: this function doesn't have any return value
    
"""
def add_data(db_name:str,table_name:str,dataframe):
    connection = make_connection()
    cursor = connection.cursor()

    cols = "`,`".join([str(i) for i in dataframe.columns.tolist()])

    # Insert DataFrame records one by one.
    for i,row in dataframe.iterrows():

        sql = f"INSERT INTO `{table_name}` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()
    connection.close()


In [9]:
add_data("nw","trip",df_to_add)

In [10]:
show_all_records("nw", "trip")

(1401, 'Prague', 9, 'POINT (14.4973794438195 50.00136875782316)', 'POINT (14.43109483523328 50.04052930943246)', datetime.datetime(2018, 5, 28, 9, 3, 40))
(1402, 'Turin', 2, 'POINT (7.672837913286881 44.9957109242058)', 'POINT (7.720368637535126 45.06782385393849)', datetime.datetime(2018, 5, 21, 2, 54, 4))
(1403, 'Prague', 8, 'POINT (14.32427345662177 50.00002074358429)', 'POINT (14.47767895969969 50.09339790740321)', datetime.datetime(2018, 5, 13, 8, 52, 25))
(1404, 'Turin', 9, 'POINT (7.541509189114433 45.09160503827746)', 'POINT (7.74528653441973 45.02628598341506)', datetime.datetime(2018, 5, 6, 9, 49, 16))
(1405, 'Turin', 12, 'POINT (7.614078119815749 45.13433106465422)', 'POINT (7.527497142312585 45.03335051325654)', datetime.datetime(2018, 5, 23, 12, 45, 54))
(1406, 'Hamburg', 9, 'POINT (10.07299025213017 53.62044974829032)', 'POINT (9.789197601249002 53.46315765148751)', datetime.datetime(2018, 5, 15, 9, 13, 36))
(1407, 'Hamburg', 13, 'POINT (9.910278201788232 53.5838626471782

In [11]:
#this function clears the entire table, so just use it for developing tasks
#clear_table_db("nw","trip")

In [12]:
min_x, min_y, max_x, max_y = (9, 50, 120, 130) #copy-paste from above just for testing
get_avg_per_bbox("nw","trip", min_x, min_y, max_x, max_y, "Hamburg")

C:\Users\carol\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


22.4